# **Analizando un grupo de WhatsApp**

## 1 Introducción

### 1.1 Configuraciones de entorno

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import spacy

import re
import zipfile
from pathlib import Path

In [2]:
plt.style.use('ggplot')

Los directorios de datos: 

In [3]:
BASE = Path().resolve().parent
DATA = BASE / "data"
RAW = DATA / "raw"
PROCESSED = DATA / "processed"

In [4]:
BASE.exists(), BASE.is_dir()

(True, True)

Aseguramos que los directorios están presentes en el proyecto: 

In [5]:
RAW.mkdir(exist_ok=True, parents=True)
PROCESSED.mkdir(exist_ok=True, parents=True)

## 2 Ingesta de Datos

Los datos han sido descargados directamente desde un gurpo de WhatsApp y colocados en `data/raw` con el nombre de `grupo.zip`. 

Verifiquemos que los datos están presentes: 

In [6]:
for item in RAW.iterdir(): 
    print(item.relative_to(BASE))

data\raw\grupo.txt
data\raw\grupo.zip


Creamos el apuntador al archivo y verificamos que existe y que es un archivo válido: 

In [7]:
zfile = RAW / "grupo.zip"

zfile.exists(), zfile.is_file()

(True, True)

El archivo de salida: 

In [8]:
outpath = RAW / "grupo.txt"

Para extraer el archivo `.zip` usamos el paquete `zipfile`: 

In [9]:
with zipfile.ZipFile(zfile, 'r') as file:
    filename = file.namelist()[0]

    with file.open(filename) as source: 
        with open(outpath, 'wb') as target:
            target.write(source.read())

Verificamos que el archivo se ha extraido correctamente: 

In [10]:
outpath.exists(), outpath.is_file()

(True, True)

Una vez extraido podemos cambiar los nombres de los participantes. En total hay 6 participantes en el gurpo y se han usado nombres de los superheroes de la liga de la justicia: Batman, Aquaman, Superman, Wonder Woman y Hulk. 

Ahora, cargamos el archivo de texto y demos un vistazo a los datos: 

In [56]:
with open(outpath, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        print(line.strip())
        if i == 10:  # las primeras 20 lineas
            break

10/4/2023, 11:39 - Los mensajes y las llamadas están cifrados de extremo a extremo. Solo las personas en este chat pueden leerlos, escucharlos o compartirlos. Obtén más información.
10/4/2023, 11:39 - ‎Wonder Woman creó el grupo "grupo".
10/4/2023, 11:39 - ‎Wonder Woman te añadió
10/4/2023, 11:40 - Flash: ✋✋✋
10/4/2023, 11:41 - Flash: Rola las fotos Wonder Woman
10/4/2023, 11:41 - Flash: Un parienton
10/4/2023, 11:43 - Wonder Woman: ayuda el Batman está hablando mayo
10/4/2023, 11:45 - Batman: Hao✋
10/4/2023, 11:55 - Flash: <Multimedia omitido>
10/4/2023, 11:55 - Flash: <Multimedia omitido>
10/4/2023, 11:55 - Flash: <Multimedia omitido>


Obtenemos una lista con las lineas del texto: 

In [57]:
with open(outpath, "r", encoding="utf-8") as f:
    lines = f.readlines()

In [58]:
len(lines), lines[:5]

(11317,
 ['10/4/2023, 11:39 - Los mensajes y las llamadas están cifrados de extremo a extremo. Solo las personas en este chat pueden leerlos, escucharlos o compartirlos. Obtén más información.\n',
  '10/4/2023, 11:39 - \u200eWonder Woman creó el grupo "grupo".\n',
  '10/4/2023, 11:39 - \u200eWonder Woman te añadió\n',
  '10/4/2023, 11:40 - Flash: ✋✋✋\n',
  '10/4/2023, 11:41 - Flash: Rola las fotos Wonder Woman\n'])

Ahora, separamos cada línea como una tupla con los campos `fecha`, `hora`, `usuario`, `mensaje`: 

In [59]:
%pdoc str.strip

Class docstring:
    Return a copy of the string with leading and trailing whitespace removed.
    
    If chars is given and not None, remove characters in chars instead.
Call docstring:
    Call self as a function.

In [60]:
import re

def get_fields(lines):
    # Regex para separar fecha, hora, usuario y mensaje
    pattern = r'(\d{1,2}/\d{1,2}/\d{4}), (\d{1,2}:\d{2}) - (.*?): (.*)'
    tuples = [] 
    
    for line in lines: 
        line = line.strip()
        match = re.match(pattern, line)
        
        if match:
            fecha = match.group(1)
            hora = match.group(2)
            nombre = match.group(3)
            contenido = match.group(4)
            tuples.append((fecha, hora, nombre, contenido))
        else:
            # Mensaje del sistema: tratamos como "usuario = <Sistema>"
            sys_pattern = r'(\d{1,2}/\d{1,2}/\d{4}), (\d{1,2}:\d{2}) - (.*)'
            sys_match = re.match(sys_pattern, line)
            
            if sys_match:
                fecha = sys_match.group(1)
                hora = sys_match.group(2)
                contenido = sys_match.group(3)
                tuples.append((fecha, hora, "<Sistema>", contenido))
            else:
                # línea inicial sin contexto
                tuples.append((None, None, "<Desconocido>", line))
    
    return tuples


In [61]:
tuple_msgs = get_fields(lines)

len(tuple_msgs), tuple_msgs[:5]

(11317,
 [('10/4/2023',
   '11:39',
   '<Sistema>',
   'Los mensajes y las llamadas están cifrados de extremo a extremo. Solo las personas en este chat pueden leerlos, escucharlos o compartirlos. Obtén más información.'),
  ('10/4/2023',
   '11:39',
   '<Sistema>',
   '\u200eWonder Woman creó el grupo "grupo".'),
  ('10/4/2023', '11:39', '<Sistema>', '\u200eWonder Woman te añadió'),
  ('10/4/2023', '11:40', 'Flash', '✋✋✋'),
  ('10/4/2023', '11:41', 'Flash', 'Rola las fotos Wonder Woman')])

Ahora, pasamos los datos de texto a `DataFrame`: 

In [62]:
df = pd.DataFrame(tuple_msgs, columns=['fecha', 'hora', 'usuario', 'mensaje'])

## 3 Limpieza y Procesamiento

Una pequeña exploración a los mensajes en forma de `DataFrame`: 

In [63]:
df.head()

,fecha,hora,usuario,mensaje
0,10/4/2023,11:39,<Sistema>,Los mensajes y las llamadas están cifrados de ...
1,10/4/2023,11:39,<Sistema>,"‎Wonder Woman creó el grupo ""grupo""."
2,10/4/2023,11:39,<Sistema>,‎Wonder Woman te añadió
3,10/4/2023,11:40,Flash,✋✋✋
4,10/4/2023,11:41,Flash,Rola las fotos Wonder Woman


In [64]:
df.dtypes

fecha      object
hora       object
usuario    object
mensaje    object
dtype: object

Es necesario unir la fecha y hora una sola columna `datetime`:

In [65]:
df['datetime'] = pd.to_datetime(df['fecha'] + ' ' + df['hora'], format="%d/%m/%Y %H:%M")
df['datetime'].head()

0   2023-04-10 11:39:00
1   2023-04-10 11:39:00
2   2023-04-10 11:39:00
3   2023-04-10 11:40:00
4   2023-04-10 11:41:00
Name: datetime, dtype: datetime64[ns]

Eliminamos las columnas `fecha` y `hora`: 

In [66]:
df.drop(columns=['fecha', 'hora'], inplace=True)

Ahora, analicemos la columna `mensaje` para determinar el tipo de mensaje: 

In [67]:
df['mensaje'].head()

0    Los mensajes y las llamadas están cifrados de ...
1                 ‎Wonder Woman creó el grupo "grupo".
2                              ‎Wonder Woman te añadió
3                                                  ✋✋✋
4                          Rola las fotos Wonder Woman
Name: mensaje, dtype: object

Conviene normalizar las cadenas de mensaje: 

In [68]:
df['mensaje'] = df['mensaje'].str.lower()

In [69]:
multimedias = df[df['mensaje'].str.contains('multimedia')]
multimedias['mensaje'].unique()

array(['<multimedia omitido>'], dtype=object)

Debido a que se han exportado los mensajes considerando únicamente los mensajes, no es posible saber si el mensaje multimedia es un audio, video, imagen, documento ó sticker. Por lo que las categirías serán unicamente `texto` y multimedia: 

In [71]:
def classify_msg(line): 
    if 'multimedia' in line:
        return 'multimedia'
    else: 
        return 'texto'

In [72]:
df['tipo'] = df['mensaje'].apply(classify_msg)

In [73]:
df['tipo'].unique()

array(['texto', 'multimedia'], dtype=object)

Ahora, se crea la columna `num_palabras` que posee la cantidad de palabras en el mensaje: 

In [75]:
df['mensaje'].head()

0    los mensajes y las llamadas están cifrados de ...
1                 ‎wonder woman creó el grupo "grupo".
2                              ‎wonder woman te añadió
3                                                  ✋✋✋
4                          rola las fotos wonder woman
Name: mensaje, dtype: object

In [84]:
df['num_palabras'] = df['mensaje'].str.split(' ').str.len()

Por último, la columna `num_chars` que almacena la cantidad de carácteres de cada mensaje: 

In [82]:
df['num_chars'] = df['mensaje'].str.len()

In [83]:
df.head()

,usuario,mensaje,datetime,tipo,num_palabras,num_chars
0,<Sistema>,los mensajes y las llamadas están cifrados de ...,2023-04-10 11:39:00,texto,25,162
1,<Sistema>,"‎wonder woman creó el grupo ""grupo"".",2023-04-10 11:39:00,texto,6,36
2,<Sistema>,‎wonder woman te añadió,2023-04-10 11:39:00,texto,4,23
3,Flash,✋✋✋,2023-04-10 11:40:00,texto,1,3
4,Flash,rola las fotos wonder woman,2023-04-10 11:41:00,texto,5,27


In [80]:
df.dtypes

usuario                 object
mensaje                 object
datetime        datetime64[ns]
tipo                    object
num_palabras             int64
dtype: object

¡La limpieza parece estar completa!

Reorganicemos las columnas del `DataFrame`: 

In [85]:
df = df[['datetime', 'usuario', 'mensaje', 'tipo', 'num_palabras', 'num_chars']]
df.head()

,datetime,usuario,mensaje,tipo,num_palabras,num_chars
0,2023-04-10 11:39:00,<Sistema>,los mensajes y las llamadas están cifrados de ...,texto,25,162
1,2023-04-10 11:39:00,<Sistema>,"‎wonder woman creó el grupo ""grupo"".",texto,6,36
2,2023-04-10 11:39:00,<Sistema>,‎wonder woman te añadió,texto,4,23
3,2023-04-10 11:40:00,Flash,✋✋✋,texto,1,3
4,2023-04-10 11:41:00,Flash,rola las fotos wonder woman,texto,5,27


## 4 Análisis Exploratorio de Datos

### 4.1 Estadísticas Descriptivas

### 4.2 Nube de palabras

### 4.3 Usuarios más activos

### 4.4 Ditribución temporal

### 4.5 Texto vs Multimedia

## 5 Otros Análsis

## 6 Conclusiones